# Python Capstone API
## SQL and Exploratory Data Analysis

**Agus Wibawa (aguscuk@gmail.com)**
- Jupyter Class
- 15 May 2020

___

In [127]:
import sqlite3
import pandas as pd
import requests

conn = sqlite3.connect("data_input/Northwind_small.sqlite") #membuat koneksi sqlite-db

northwind_table = pd.read_sql_query("SELECT * \
                FROM sqlite_master \
                WHERE type ='table';", conn) #query
northwind_table

,type,name,tbl_name,rootpage,sql
0,table,Employee,Employee,2,"CREATE TABLE ""Employee"" \n(\n ""Id"" INTEGER PR..."
1,table,Category,Category,3,"CREATE TABLE ""Category"" \n(\n ""Id"" INTEGER PR..."
2,table,Customer,Customer,4,"CREATE TABLE ""Customer"" \n(\n ""Id"" VARCHAR(80..."
3,table,Shipper,Shipper,8,"CREATE TABLE ""Shipper"" \n(\n ""Id"" INTEGER PRI..."
4,table,Supplier,Supplier,9,"CREATE TABLE ""Supplier"" \n(\n ""Id"" INTEGER PR..."
5,table,Order,Order,11,"CREATE TABLE ""Order"" \n(\n ""Id"" INTEGER PRIMA..."
6,table,Product,Product,12,"CREATE TABLE ""Product"" \n(\n ""Id"" INTEGER PRI..."
7,table,OrderDetail,OrderDetail,14,"CREATE TABLE ""OrderDetail"" \n(\n ""Id"" VARCHAR..."
8,table,CustomerCustomerDemo,CustomerCustomerDemo,16,"CREATE TABLE ""CustomerCustomerDemo"" \n(\n ""Id..."
9,table,CustomerDemographic,CustomerDemographic,18,"CREATE TABLE ""CustomerDemographic"" \n(\n ""Id""..."


The Northwind sample database was provided with Microsoft as a tutorial schema for managing small business customers, orders, inventory, purchasing, suppliers, shipping, and employees. Northwind is an excellent tutorial schema for a small-business ERP, with customers, orders, inventory, purchasing, suppliers, shipping, employees, and single-entry accounting

![](assets/Northwind_ERD.png)

In [128]:
# query untuk produk

products = pd.read_sql_query("SELECT P.Id ProductId, \
       P.ProductName, \
       P.QuantityPerUnit, \
       P.UnitPrice, \
       P.UnitsInStock, \
       P.ReorderLevel, \
       P.Discontinued, \
       S.CompanyName SupplierName, \
       S.ContactName, \
       S.Address, \
       S.City, \
       S.Region, \
       S.Phone, \
       S.Fax \
  FROM Product P \
       LEFT JOIN \
       Supplier S ON P.SupplierId = S.Id \
       LEFT JOIN \
       Category C ON P.CategoryId = C.Id;", conn, index_col='ProductId')

products.tail()

,ProductName,QuantityPerUnit,UnitPrice,UnitsInStock,ReorderLevel,Discontinued,SupplierName,ContactName,Address,City,Region,Phone,Fax
ProductId,,,,,,,,,,,,,
73,Röd Kaviar,24 - 150 g jars,15.00,101,5,0,Svensk Sjöföda AB,Michael Björn,Brovallavägen 231,Stockholm,Northern Europe,08-123 45 67,None
74,Longlife Tofu,5 kg pkg.,10.00,4,5,0,Tokyo Traders,Yoshi Nagase,9-8 Sekimai Musashino-shi,Tokyo,Eastern Asia,(03) 3555-5011,None
75,Rhönbräu Klosterbier,24 - 0.5 l bottles,7.75,125,25,0,Plutzer Lebensmittelgroßmärkte AG,Martin Bein,Bogenallee 51,Frankfurt,Western Europe,(069) 992755,None
76,Lakkalikööri,500 ml,18.00,57,20,0,Karkki Oy,Anne Heikkonen,Valtakatu 12,Lappeenranta,Scandinavia,(953) 10956,None
77,Original Frankfurter grüne Soße,12 boxes,13.00,32,15,0,Plutzer Lebensmittelgroßmärkte AG,Martin Bein,Bogenallee 51,Frankfurt,Western Europe,(069) 992755,None


In [129]:
# sql query untuk customer

customers = pd.read_sql_query("SELECT * FROM Customer;", conn)
customers.head()

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,Western Europe,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,Central America,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,Central America,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,British Isles,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,Northern Europe,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


In [130]:
# query orders join table dengan OrderDetail, Product, Customer dan Category

orders = pd.read_sql_query("SELECT O.Id OrderId, \
       O.OrderDate, \
       O.RequiredDate, \
       O.ShippedDate, \
       O.ShipRegion, \
       O.ShipCountry, \
       Od.UnitPrice, \
       Od.Quantity, \
       Od.Discount, \
       P.ProductName, \
       ROUND(Od.UnitPrice * Od.Quantity, 2) AS SubTotal, \
       (Od.UnitPrice * Od.Quantity * Od.Discount) AS DiscPrice, \
       (Od.UnitPrice * Od.Quantity) - (Od.UnitPrice * Od.Quantity * Od.Discount) AS Total, \
       Cu.CompanyName AccountName, \
       Cu.ContactName, \
       Cu.ContactTitle, \
       Cu.Address, \
       Cu.City, \
       Cu.Region, \
       Cu.PostalCode, \
       Cu.Country, \
       Cu.Phone, \
       Cu.Fax, \
       Ca.CategoryName, \
       Ca.Description \
  FROM [Order] O \
       LEFT JOIN \
       OrderDetail Od ON O.Id = Od.OrderId \
       LEFT JOIN \
       Product P ON Od.ProductId = P.Id \
       LEFT JOIN \
       Customer Cu ON O.CustomerId = Cu.Id \
       LEFT JOIN \
       Category Ca ON P.CategoryId = Ca.Id;" \
       , conn, parse_dates=["OrderDate", "RequiredDate", "ShippedDate"], index_col = "OrderId")

orders.head()

,OrderDate,RequiredDate,ShippedDate,ShipRegion,ShipCountry,UnitPrice,Quantity,Discount,ProductName,SubTotal,...,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,CategoryName,Description
OrderId,,,,,,,,,,,,,,,,,,,,,
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,14.0,12,0.0,Queso Cabrales,168.0,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,9.8,10,0.0,Singaporean Hokkien Fried Mee,98.0,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Grains/Cereals,"Breads, crackers, pasta, and cereal"
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,34.8,5,0.0,Mozzarella di Giovanni,174.0,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,18.6,9,0.0,Tofu,167.4,...,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,42.4,40,0.0,Manjimup Dried Apples,1696.0,...,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd


In [131]:
# periksa tipe data masing2 field

orders.dtypes

OrderDate       datetime64[ns]
RequiredDate    datetime64[ns]
ShippedDate     datetime64[ns]
ShipRegion              object
ShipCountry             object
UnitPrice              float64
Quantity                 int64
Discount               float64
ProductName             object
SubTotal               float64
DiscPrice              float64
Total                  float64
AccountName             object
ContactName             object
ContactTitle            object
Address                 object
City                    object
Region                  object
PostalCode              object
Country                 object
Phone                   object
Fax                     object
CategoryName            object
Description             object
dtype: object

In [132]:
# ubah tipe data object beberapa field yang perlu menjadi Categorical

orders[['ShipRegion', 'ShipCountry', 'ProductName', 'City', 'Region', 'Country', 'CategoryName']] = orders[['ShipRegion', 'ShipCountry', 'ProductName' , 'City', 'Region', 'Country', 'CategoryName']].astype('category')
orders.dtypes

OrderDate       datetime64[ns]
RequiredDate    datetime64[ns]
ShippedDate     datetime64[ns]
ShipRegion            category
ShipCountry           category
UnitPrice              float64
Quantity                 int64
Discount               float64
ProductName           category
SubTotal               float64
DiscPrice              float64
Total                  float64
AccountName             object
ContactName             object
ContactTitle            object
Address                 object
City                  category
Region                category
PostalCode              object
Country               category
Phone                   object
Fax                     object
CategoryName          category
Description             object
dtype: object

In [133]:
# check hasilnya
orders.head()

,OrderDate,RequiredDate,ShippedDate,ShipRegion,ShipCountry,UnitPrice,Quantity,Discount,ProductName,SubTotal,...,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,CategoryName,Description
OrderId,,,,,,,,,,,,,,,,,,,,,
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,14.0,12,0.0,Queso Cabrales,168.0,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,9.8,10,0.0,Singaporean Hokkien Fried Mee,98.0,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Grains/Cereals,"Breads, crackers, pasta, and cereal"
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,34.8,5,0.0,Mozzarella di Giovanni,174.0,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,18.6,9,0.0,Tofu,167.4,...,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,42.4,40,0.0,Manjimup Dried Apples,1696.0,...,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd


In [134]:
# periksa banyak data
orders.shape

(2155, 24)

In [135]:
orders['OrderDate'].dt.day_name()
orders['OrderDate'].dt.week
orders['OrderDate'].dt.month
orders['OrderDate'].dt.year

OrderId
10248    2012
10248    2012
10248    2012
10249    2012
10249    2012
         ... 
11077    2014
11077    2014
11077    2014
11077    2014
11077    2014
Name: OrderDate, Length: 2155, dtype: int64

In [136]:
# extaract field OrderDate menjadi OrderDate_year / month / week / day_name 

orders['OrderDate_dayname'] = orders['OrderDate'].dt.day_name()
orders['OrderDate_week'] = orders['OrderDate'].dt.week
orders['OrderDate_month'] = orders['OrderDate'].dt.month
orders['OrderDate_year'] = orders['OrderDate'].dt.year
orders['OrderDate_quarter'] = orders['OrderDate'].dt.to_period('Q')

In [137]:
# khusus untuk OrderDate_dayname kita urutkan berdasarkan urutan hari

dayorder = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
orders['OrderDate_dayname'] = pd.Categorical(orders['OrderDate_dayname'],
                                         categories=dayorder,
                                         ordered=True)

In [138]:
orders.head()

,OrderDate,RequiredDate,ShippedDate,ShipRegion,ShipCountry,UnitPrice,Quantity,Discount,ProductName,SubTotal,...,Country,Phone,Fax,CategoryName,Description,OrderDate_dayname,OrderDate_week,OrderDate_month,OrderDate_year,OrderDate_quarter
OrderId,,,,,,,,,,,,,,,,,,,,,
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,14.0,12,0.0,Queso Cabrales,168.0,...,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses,Wednesday,27,7,2012,2012Q3
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,9.8,10,0.0,Singaporean Hokkien Fried Mee,98.0,...,France,26.47.15.10,26.47.15.11,Grains/Cereals,"Breads, crackers, pasta, and cereal",Wednesday,27,7,2012,2012Q3
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,34.8,5,0.0,Mozzarella di Giovanni,174.0,...,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses,Wednesday,27,7,2012,2012Q3
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,18.6,9,0.0,Tofu,167.4,...,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd,Thursday,27,7,2012,2012Q3
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,42.4,40,0.0,Manjimup Dried Apples,1696.0,...,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd,Thursday,27,7,2012,2012Q3


In [139]:
orders.dtypes

OrderDate            datetime64[ns]
RequiredDate         datetime64[ns]
ShippedDate          datetime64[ns]
ShipRegion                 category
ShipCountry                category
UnitPrice                   float64
Quantity                      int64
Discount                    float64
ProductName                category
SubTotal                    float64
DiscPrice                   float64
Total                       float64
AccountName                  object
ContactName                  object
ContactTitle                 object
Address                      object
City                       category
Region                     category
PostalCode                   object
Country                    category
Phone                        object
Fax                          object
CategoryName               category
Description                  object
OrderDate_dayname          category
OrderDate_week                int64
OrderDate_month               int64
OrderDate_year              

In [140]:
# check missing value

orders.isna().sum()

OrderDate              0
RequiredDate           0
ShippedDate           73
ShipRegion             0
ShipCountry            0
UnitPrice              0
Quantity               0
Discount               0
ProductName            0
SubTotal               0
DiscPrice              0
Total                  0
AccountName           77
ContactName           77
ContactTitle          77
Address               77
City                  77
Region                77
PostalCode           132
Country               77
Phone                 77
Fax                  669
CategoryName           0
Description            0
OrderDate_dayname      0
OrderDate_week         0
OrderDate_month        0
OrderDate_year         0
OrderDate_quarter      0
dtype: int64

In [141]:
orders.shape

(2155, 29)

Dari pemeriksaan data missing value, data tidak ada yang perlu di drop, karena missing value pada beberapa field, seperti ShippedDate, AccountName, ContactName dll tidak berpengaruh pada analisa.

In [150]:
# orders omzet top5 by countries 

top5_orders = pd.read_sql_query("SELECT \
       Od.UnitPrice, \
       Od.Quantity, \
       Od.Discount, \
       SUM((Od.UnitPrice * Od.Quantity) - (Od.UnitPrice * Od.Quantity * Od.Discount)) AS Total, \
       Cu.Country \
  FROM [Order] O \
       LEFT JOIN \
       OrderDetail Od ON O.Id = Od.OrderId \
       LEFT JOIN \
       Product P ON Od.ProductId = P.Id \
       LEFT JOIN \
       Customer Cu ON O.CustomerId = Cu.Id \
       LEFT JOIN \
       Category Ca ON P.CategoryId = Ca.Id \
  GROUP BY Cu.Country \
  ORDER BY Total DESC;" \
  , conn)

top5_orders.head(5)

,UnitPrice,Quantity,Discount,Total,Country
0,17.0,12,0.2,245584.6105,USA
1,18.6,9,0.0,230284.6335,Germany
2,15.2,50,0.2,128003.8385,Austria
3,7.7,10,0.0,81208.2790,Brazil
4,14.0,12,0.0,79742.4225,France


In [155]:
# jadikan 5 countries menjadi list

top5 = top5_orders.groupby('Country').Total.sum().sort_values(ascending=False).head().index.to_list()
top5


['USA', 'Germany', 'Austria', 'Brazil', 'France']

In [156]:
# dari list top5 countries, filter dataframe orders seseuai yang termasuk top5  

top5_data = orders[orders['Country'].isin(top5)].copy()
top5_data.dtypes

OrderDate            datetime64[ns]
RequiredDate         datetime64[ns]
ShippedDate          datetime64[ns]
ShipRegion                 category
ShipCountry                category
UnitPrice                   float64
Quantity                      int64
Discount                    float64
ProductName                category
SubTotal                    float64
DiscPrice                   float64
Total                       float64
AccountName                  object
ContactName                  object
ContactTitle                 object
Address                      object
City                       category
Region                     category
PostalCode                   object
Country                    category
Phone                        object
Fax                          object
CategoryName               category
Description                  object
OrderDate_dayname          category
OrderDate_week                int64
OrderDate_month               int64
OrderDate_year              

In [158]:
top5_data.head()

,OrderDate,RequiredDate,ShippedDate,ShipRegion,ShipCountry,UnitPrice,Quantity,Discount,ProductName,SubTotal,...,Country,Phone,Fax,CategoryName,Description,OrderDate_dayname,OrderDate_week,OrderDate_month,OrderDate_year,OrderDate_quarter
OrderId,,,,,,,,,,,,,,,,,,,,,
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,14.0,12,0.0,Queso Cabrales,168.0,...,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses,Wednesday,27,7,2012,2012Q3
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,9.8,10,0.0,Singaporean Hokkien Fried Mee,98.0,...,France,26.47.15.10,26.47.15.11,Grains/Cereals,"Breads, crackers, pasta, and cereal",Wednesday,27,7,2012,2012Q3
10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,34.8,5,0.0,Mozzarella di Giovanni,174.0,...,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses,Wednesday,27,7,2012,2012Q3
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,18.6,9,0.0,Tofu,167.4,...,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd,Thursday,27,7,2012,2012Q3
10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,42.4,40,0.0,Manjimup Dried Apples,1696.0,...,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd,Thursday,27,7,2012,2012Q3


In [159]:
# kemudian dicari omzet Total order tersebut berdasarkan nama hari dari top5 countries

pivot_top5_data = pd.pivot_table(
    data=top5_data,
    index='OrderDate_dayname',
    columns=['Country'],
    values='Total',
    aggfunc='sum',
    margins = True
)

pivot_top5_data

Country,Austria,Brazil,France,Germany,USA,All
OrderDate_dayname,,,,,,
Monday,23868.7875,18291.770,13575.8500,51116.4370,50406.5475,157259.3920
Tuesday,32993.1600,10665.290,15848.1725,45434.7450,41605.6355,146547.0030
Wednesday,26798.0435,8087.089,11971.6400,31135.2125,38430.0950,116422.0800
Thursday,18354.9675,28136.100,26639.4400,45566.6590,59739.2200,178436.3865
Sunday,25988.8800,16028.030,11707.3200,57031.5800,55403.1125,166158.9225
All,128003.8385,81208.279,79742.4225,230284.6335,245584.6105,764823.7840


## API Flask

In [3]:
url1 = 'http://localhost:5000/api/v1/resources/products/all'
r = requests.get(url1)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,ProductName,QuantityPerUnit,UnitPrice,UnitsInStock,ReorderLevel,Discontinued,SupplierName,ContactName,Address,City,Region,Phone,Fax
1,Chai,10 boxes x 20 bags,18.00,39,10,0,Exotic Liquids,Charlotte Cooper,49 Gilbert St.,London,British Isles,(171) 555-2222,None
2,Chang,24 - 12 oz bottles,19.00,17,25,0,Exotic Liquids,Charlotte Cooper,49 Gilbert St.,London,British Isles,(171) 555-2222,None
3,Aniseed Syrup,12 - 550 ml bottles,10.00,13,25,0,Exotic Liquids,Charlotte Cooper,49 Gilbert St.,London,British Isles,(171) 555-2222,None
4,Chef Anton's Cajun Seasoning,48 - 6 oz jars,22.00,53,0,0,New Orleans Cajun Delights,Shelley Burke,P.O. Box 78934,New Orleans,North America,(100) 555-4822,None
5,Chef Anton's Gumbo Mix,36 boxes,21.35,0,0,1,New Orleans Cajun Delights,Shelley Burke,P.O. Box 78934,New Orleans,North America,(100) 555-4822,None


In [4]:
url1 = 'http://localhost:5000/api/v1/resources/suppliers/all'
r = requests.get(url1)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,Id,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,HomePage
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,British Isles,EC1 4SD,UK,(171) 555-2222,None,None
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,North America,70117,USA,(100) 555-4822,None,#CAJUN.HTM#
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,North America,48104,USA,(313) 555-5735,(313) 555-3349,None
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,Eastern Asia,100,Japan,(03) 3555-5011,None,None
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Southern Europe,33007,Spain,(98) 598 76 54,None,None


In [6]:
import requests

url1 = 'http://localhost:5000/api/v1/resources/orders/all'
r = requests.get(url1)
r_pd = pd.DataFrame(r.json())
r_pd.head()

,OrderId,OrderDate,RequiredDate,ShippedDate,ShipRegion,ShipCountry,UnitPrice,Quantity,Discount,ProductName,...,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax,CategoryName,Description
0,10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,14.0,12,0.0,Queso Cabrales,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses
1,10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,9.8,10,0.0,Singaporean Hokkien Fried Mee,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Grains/Cereals,"Breads, crackers, pasta, and cereal"
2,10248,2012-07-04,2012-08-01,2012-07-16,Western Europe,France,34.8,5,0.0,Mozzarella di Giovanni,...,Accounting Manager,59 rue de l'Abbaye,Reims,Western Europe,51100,France,26.47.15.10,26.47.15.11,Dairy Products,Cheeses
3,10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,18.6,9,0.0,Tofu,...,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd
4,10249,2012-07-05,2012-08-16,2012-07-10,Western Europe,Germany,42.4,40,0.0,Manjimup Dried Apples,...,Marketing Manager,Luisenstr. 48,Münster,Western Europe,44087,Germany,0251-031259,0251-035695,Produce,Dried fruit and bean curd
